In [ ]:
# Sentence -> word -> creat pair sequence of two words by array list -> given word(input) for prediction and prdict output(corresponding next word) 

In [ ]:
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

## Source Text 

In [ ]:
data = """Shakib Al Hasan is a Bangladeshi international cricketer. He is considered to be one of the greatest all-rounders of all time.\n
He was ranked as one of the world's most famous athletes by ESPN World Fame 100 in 2019. His aggressive left-handed batting style in the middle order,\n
controlled slow left-arm orthodox bowling has made him a consistent player for Bangladesh. Shakib is the highest wicket taker for Bangladesh in all international formats.\n
In August 2021, he became the first cricketer to have the double of 100 wickets and 1,000 runs in T20Is history. He is now the only player in the world,\n
who have the double of 100 wickets and 1000 runs feat, in all three formats of men's cricket."""
data

## Encoded text

In [ ]:
# Sentence -> word -> encode words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded_data = tokenizer.texts_to_sequences([data])[0]
encoded_data

In [ ]:
# Vocabulary size
vocab_size = len(tokenizer.word_index) + 1   # 0 is reserved for padding so that's why added 1 | don't count start from 0 
vocab_size

## Create Sequences

In [ ]:
Sequences = list()
for i in range(1, len(encoded_data)):   # goes upto encoded data(79)
    sequence = encoded_data[i-1:i+1]
    Sequences.append(sequence)    # store in list
print('Total Sequences: %d' % len(Sequences))
print(Sequences)   # [to, be] -> [one, ]| we have total 124 input-output pairs

## Split train and test

In [ ]:
# [input, output] | input & output are the columns
Sequences = array(Sequences)
X, y = Sequences[:,0], Sequences[:,1]
print(X[:5])
print(y[:5])   

In [ ]:
# One Hot Encoding for y column
y = to_categorical(y, num_classes = vocab_size)
y[:5]   # after one hot encoding for 5 values

## LSTM model

In [ ]:
mymodel = Sequential()
mymodel.add(Embedding(vocab_size, 10, input_length = 1))
mymodel.add(LSTM(30))
mymodel.add(Dense(vocab_size, activation = 'softmax'))
print(mymodel.summary())

In [ ]:
mymodel.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
mymodel.fit(X, y, epochs = 30)

In [ ]:
def generate_seq(mymodel, tokenizer, enter_text, n_pred):
    in_text, result = enter_text, enter_text   # both in_text & result variable store given text
    
    for _ in range(n_pred):    # how many times for loops run
        
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        encoded = array(encoded)
        
        # predict a word in the vocabulary
        yHat = mymodel.predict_classes(encoded)
        
        # 
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yHat:
                out_word = word
                break
        in_text, result = out_word, result + ' ' + out_word
    return result

In [ ]:
# predict output
print(generate_seq(mymodel, tokenizer, 'the', 1))    # also print input word | next n predicted word